In [1]:
import pandas as pd


In [2]:
data_path = '../datasets/preprocessing/'
activity_df = pd.read_csv(data_path+'activity_final.csv')
sleep_df = pd.read_csv(data_path+'sleep_final.csv')
mmse_df = pd.read_csv(data_path+'mmse_final.csv')

In [3]:
activity_df.head()

,activity_cal_total,activity_high,activity_inactive,activity_low,activity_medium,activity_met_min_inactive,activity_non_wear,activity_rest,activity_score,activity_score_meet_daily_targets,activity_score_move_every_hour,activity_score_recovery_time,activity_score_training_frequency,activity_score_training_volume,activity_steps,activity_total,DIAG_NM
0,0.234154,0.023222,0.620100,0.149217,0.087749,0.290130,0.201264,0.516628,0.209601,0.140492,0.634341,0.996484,0.163528,0.403653,0.106811,0.138529,0
1,0.000000,0.007217,0.531848,0.460230,0.071337,0.704377,0.243731,0.343130,0.125933,0.064371,0.784069,0.955738,0.000000,0.321114,0.180278,0.362767,1
2,0.318593,0.018581,0.657863,0.569825,0.035947,0.936819,0.155918,0.186934,0.351903,0.513854,0.861448,0.998392,0.113332,0.258988,0.158658,0.433723,0
3,0.393307,0.010964,0.444411,0.604473,0.158410,0.493903,0.178951,0.291086,0.864838,0.916883,0.957027,0.979426,0.826441,0.911869,0.295948,0.496304,1
4,0.183541,0.060878,0.334683,0.564425,0.143397,0.617737,0.664560,0.292612,0.618191,0.709387,0.930424,0.998668,0.397923,0.521875,0.210219,0.464942,0


In [4]:
sleep_df = sleep_df.drop(columns='EMAIL')

In [5]:
sleep_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 141 entries, 0 to 140
Data columns (total 19 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   sleep_awake               141 non-null    float64
 1   sleep_breath_average      141 non-null    float64
 2   sleep_deep                141 non-null    float64
 3   sleep_duration            141 non-null    float64
 4   sleep_hr_average          141 non-null    float64
 5   sleep_light               141 non-null    float64
 6   sleep_midpoint_at_delta   141 non-null    float64
 7   sleep_onset_latency       141 non-null    float64
 8   sleep_period_id           141 non-null    float64
 9   sleep_rem                 141 non-null    float64
 10  sleep_restless            141 non-null    float64
 11  sleep_rmssd               141 non-null    float64
 12  sleep_score               141 non-null    float64
 13  sleep_score_alignment     141 non-null    float64
 14  sleep_scor

In [6]:
mmse_df = mmse_df.drop(columns='SAMPLE_EMAIL')

In [7]:
mmse_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 141 entries, 0 to 140
Data columns (total 32 columns):
 #   Column   Non-Null Count  Dtype
---  ------   --------------  -----
 0   DIAG_NM  141 non-null    int64
 1   Q01      141 non-null    int64
 2   Q02      141 non-null    int64
 3   Q03      141 non-null    int64
 4   Q04      141 non-null    int64
 5   Q05      141 non-null    int64
 6   Q06      141 non-null    int64
 7   Q07      141 non-null    int64
 8   Q08      141 non-null    int64
 9   Q09      141 non-null    int64
 10  Q10      141 non-null    int64
 11  Q11_1    141 non-null    int64
 12  Q11_2    141 non-null    int64
 13  Q11_3    141 non-null    int64
 14  Q12_1    141 non-null    int64
 15  Q12_2    141 non-null    int64
 16  Q12_3    141 non-null    int64
 17  Q12_4    141 non-null    int64
 18  Q12_5    141 non-null    int64
 19  Q13_1    141 non-null    int64
 20  Q13_2    141 non-null    int64
 21  Q13_3    141 non-null    int64
 22  Q14_1    141 non-null    i

학습데이터 생성 1) activity

In [8]:
train_num = int(len(activity_df)*0.8)
print(train_num)

112


In [9]:
x_df = activity_df.drop(['DIAG_NM'], axis=1)
print('x:', len(x_df))

y_df = activity_df['DIAG_NM']
print('y:', len(y_df))

x: 141
y: 141


In [10]:
print(activity_df['DIAG_NM'].value_counts())

DIAG_NM
0    85
1    56
Name: count, dtype: int64


In [11]:
train_X = x_df[:train_num].values
train_y = y_df[:train_num].values
test_X = x_df[train_num:].values
test_y = y_df[train_num:].values

In [13]:
def split_train_test(dataframe, target_colum:str, percent:float = 0.8):
    train_num = int(len(dataframe)*percent)
    x_df = activity_df.drop([target_colum], axis=1)
    y_df = activity_df[target_colum]
    train_X = x_df[:train_num].values
    train_y = y_df[:train_num].values
    test_X = x_df[train_num:].values
    test_y = y_df[train_num:].values
    return train_X, train_y, test_X, test_y

사이킷런 래퍼 XGBoost 클래스인 XGBClassifier 임포트

In [12]:
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import roc_auc_score

def gef_clf_eval(y_test, pred=None, pred_proba=None):
    confusion = confusion_matrix(y_test, pred)
    accuracy = accuracy_score(y_test, pred)
    precision = precision_score(y_test, pred)
    recall = recall_score(y_test, pred)
    f1 = f1_score(y_test, pred)
    
    # ROC_AUC 추가
    roc_auc = roc_auc_score(y_test, pred_proba)
    print('오차 행렬')
    print(confusion)
    # ROC_AUC print 추가
    print('정확도:{0:.4f}, 정밀도:{1:.4f}, 재현율:{2:.4f}, F1:{3:.4f}, AUC:{4:.4f}'.format(accuracy, precision, recall, f1, roc_auc))

In [14]:
sleep_df.head()

,sleep_awake,sleep_breath_average,sleep_deep,sleep_duration,sleep_hr_average,sleep_light,sleep_midpoint_at_delta,sleep_onset_latency,sleep_period_id,sleep_rem,sleep_restless,sleep_rmssd,sleep_score,sleep_score_alignment,sleep_score_deep,sleep_score_disturbances,sleep_score_latency,sleep_temperature_delta,DIAG_NM
0,0.422924,0.477566,0.304000,0.523079,0.460202,0.423397,0.261079,0.261016,0.685649,0.449271,0.599543,0.161833,0.415818,0.576342,0.675137,0.336553,0.648496,0.599968,0
1,0.000000,0.451455,0.488372,0.501219,0.637767,0.477082,0.196890,0.239434,0.159950,0.710106,0.334438,0.435383,0.822495,0.778833,0.859601,0.949620,0.853174,0.665333,1
2,0.430326,0.555427,0.182253,0.404442,0.343454,0.250976,0.096846,0.375251,0.139814,0.525399,0.443120,0.237821,0.365140,0.965204,0.489051,0.481131,0.583690,0.473578,0
3,0.379785,0.195751,0.308392,0.442231,0.592405,0.385784,0.190902,0.190483,0.350861,0.289051,0.738342,0.232490,0.382775,0.911952,0.693454,0.286148,0.796513,0.317933,1
4,0.256004,0.347874,0.677276,0.438594,0.360021,0.331052,0.142572,0.179954,0.157162,0.081636,0.438556,0.708183,0.541966,0.815481,0.955445,0.556310,0.500417,1.000000,0


In [22]:
from xgboost import XGBClassifier

train_X, train_y, test_X, test_y = split_train_test(sleep_df, 'DIAG_NM')

model_xgb = XGBClassifier(n_estimators=500, learning_rate=0.3, max_depth=4, random_state=32)
model_xgb.fit(train_X, train_y, verbose=True)
preds = model_xgb.predict(test_X)
preds_proba = model_xgb.predict_proba(test_X)[:,1]

In [26]:
test_y

array([1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 1, 0, 1])

In [23]:
gef_clf_eval(test_y, preds, preds_proba)

오차 행렬
[[12  6]
 [ 8  3]]
정확도:0.5172, 정밀도:0.3333, 재현율:0.2727, F1:0.3000, AUC:0.5303
